In [80]:
# General setup

import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

**Read the training data file:**

In [81]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

**Read the test data file:**

In [82]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

In [83]:
sum(train_data["Survived"])/len(train_data["Survived"])

## **Check the distribution of people by age and how many of them survived:**


#### Hipothesys
The hypothesis is that younger travelers has more chances to survive to the disaster than the old ones. 

#### Findings
We've discovered that most of the part of the travelers were younger than 30 years and 90% younger than 50 years, althougt there was one with 80 years. 

It's important to detail that part of the data about the travelers age was missing: 177 people didn't have an age informed. In this case, we chose to replace these missing values for de mean of the travelers ages.

#### Conclusion
As a conclusion,the 10% youngest people had the highest chances to survive (59.04%), and young adults (20 to 24 years) had the lowest (28.92%). The hypothesis was partially correct, as the  10% and 20% oldest people had a survival rate of 36.67% and 38%. from 20 to 47 years, there was no visible relationship between the age and the survival rate, as all of the groups had a survival rate between 31% and 42% - except for young adults (28.92%) and 32-38 years (45%).

If our traveler has between 20 and 47 years, the age is not the best metrics to check if he will or not survive.

In [84]:
pd.Series(train_data['Age']).describe()
percentages = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

print(f'Total of missing age values: {train_data["Age"].isna().sum()}')

test_data["Age"] = test_data["Age"].fillna(test_data["Age"].mean().round())
train_data["Age"] = train_data["Age"].fillna(train_data["Age"].mean().round())

age_percentiles = pd.Series(train_data['Age']).quantile(percentages)

for perc in percentages:
    survived_by_perc = train_data.loc[train_data.Age<pd.Series(train_data['Age']).quantile(perc)]["Survived"]
    adjust = train_data.loc[train_data.Age<pd.Series(train_data['Age']).quantile(perc-0.1)]["Survived"]
    total_people_by_perc = len(survived_by_perc) - len(adjust)
    survived_people_by_perc = sum(survived_by_perc) - sum(adjust)
    if total_people_by_perc != 0:
        perc_survived_by_perc = round((survived_people_by_perc/total_people_by_perc)*100, 2)
        print(f'From the {(perc*100)-10}% to {perc*100}% youngest - until {int(age_percentiles[perc])} years -, {survived_people_by_perc} people survived, from a total of {total_people_by_perc} - {perc_survived_by_perc}%')


**Is embarked relevant?**


#### Hypothesis
There is no relevant relationship between the port of embakation and the survival rate. 

#### Findings

Let's check if the Port of Embarkation (Cherbourg, Queenstown or Southampton) is relevant. We found that 55% of Cherbourg passengers had survive, altought the travelers from Queenstown and Southampton had a survival rate of 38.9% and 33.6%. 

#### Conclusion 
The Cherbourg has a higher number of survivors, altought that Queenstown and Southampton has similar surviving rates. From a first approach, the port of embarkations seems to be a relevant metric to check the chacnes of the traveler to survive.

In [85]:
embarked_C = train_data.loc[train_data.Embarked == 'C']['Survived']
rate_embarked_C = sum(embarked_C)/len(embarked_C)

embarked_Q = train_data.loc[train_data.Embarked == 'Q']['Survived']
rate_embarked_Q = sum(embarked_Q)/len(embarked_Q)

embarked_S = train_data.loc[train_data.Embarked == 'S']['Survived']
rate_embarked_S = sum(embarked_S)/len(embarked_S)

print(f'Cherbourg: from a total of {len(embarked_C)} people, {rate_embarked_C*100}% survived')
print(f'Queenstown: from a total of {len(embarked_Q)} people, {rate_embarked_Q*100}% survived')
print(f'Southampton: from a total of {len(embarked_S)} people, {rate_embarked_S*100}% survived')

embarked_C = train_data.loc[train_data.Embarked == 'C']
embarked_C

**Relevance of ticket class**

#### Hypothesis
Money is power, so it's possible that the ticket class of the traveler can be a relevant metric to check if he will or not survive. As in the recent air travels you may pay more to get the emergency exit sit, there is a big chance that the proportional biggest part of the survivors bought tickets of first class.

#### Findings
63% of the first class travelers survived, against 47% of second class travelers and just 24% of third class. In number of travelers, 3rd class had more than the sum of 1st and 2nd classes.

#### Conclusion
As we expected, money is power. Almost 2/3 of travelers from the 1st class did survived, agains the 24% from the 3rd class. So the class is a relevant variable to determine the survival rate.

In [86]:
ticket_1 = train_data.loc[train_data.Pclass == 1]['Survived']
rate_ticket_1 = sum(ticket_1)/len(ticket_1)

ticket_2 = train_data.loc[train_data.Pclass == 2]['Survived']
rate_ticket_2 = sum(ticket_2)/len(ticket_2)

ticket_3 = train_data.loc[train_data.Pclass == 3]['Survived']
rate_ticket_3 = sum(ticket_3)/len(ticket_3)

print(f'1st class: from a total of {len(ticket_1)} people, {rate_ticket_1*100}% survived')
print(f'2nd class: from a total of {len(ticket_2)} people, {rate_ticket_2*100}% survived')
print(f'3rd class: from a total of {len(ticket_3)} people, {rate_ticket_3*100}% survived')

**Relevance of gender**

#### Hypothesis
Is gender relevant for survival purposals? We think not, even with Jack and Rose situation in Titanic movie.

#### Findings
The majority of the passengers are male (577 men against 314 women), and 18.8% of men survived, against 74.2% of women.

#### Conclusion
Data shows that there is a relevant distinction between the survival rates for men and women. So gender is really relevant to determine if the traveler will or not survive. In this case, women have more than 3 times of chances to survive to the disaster.

In [87]:
women = train_data.loc[train_data.Sex=="female"]["Survived"]
rate_women = sum(women)/len(women)
rate_women

men = train_data.loc[train_data.Sex=="male"]["Survived"]
rate_men = sum(men)/len(men)
rate_men

print(f'From {len(men)} men, {rate_men*100}% survived. From {len(women)} women, {rate_women*100}% survived.')

**Relevance of parent/child relation**

#### Hypothesis
According to previous tests, a good rate of children survived to the disaster. Based on this, we can assume that most part of these children was traveling with their parents, and as the kids survived, we supose that the parents did to - you should not put a children alone in a lifeboat full of stranger preople.

#### Findings

After the test, we found that 34% of people traveling without children/parents survived, agains more than 50% of people with children or parents. 

#### Conclusion
So there is a possibility that this metric is relevant to determine if the travaler will or not survive. 

In [88]:
train_data.Parch.value_counts(normalize=True)*100

no_parch_rate = sum(train_data.loc[train_data.Parch==0]['Survived'])/len(train_data.loc[train_data.Parch==0]['Survived'])
parch_rate = sum(train_data.loc[train_data.Parch>0]['Survived'])/len(train_data.loc[train_data.Parch>0]['Survived'])

print(f'{no_parch_rate*100}% of people with no parents/children survived, and  {parch_rate*100}% of people with parents/children survived')

**Sibling/spouse relationship**

#### Hypothesis
One of the possible metrics to determine the survival rate is the number of spouses of siblings of the passenger. We think that this is not a relevant metric to determine the survival rate. 

#### Findings
Almost 70% of people was traveling alone (no siblings or spouses). And from these total, 34% did survive. 

209 people were traveling with only one sibling or spouse, and 53.5% of these survived. This is an interesting number.

There was a little number of passengers traveling with 2 or more siblings/spouses, but is interesting to find that people with 2 siblings/spouses had a good survival rate (46.4%), and people with 3 or more siblings/spouses had low survival rates (from zero to 25%). People with 5 or more siblings/spouses (there was a small number of examples) did not survive.

#### Conclusion
Our hypothesis was incorrect: if you were traveling with only one sibling or spouse, you probably should survive to the disaster. People traveling alone kept very close to the general survival rate of the Titanic (38% of all passengers survived). Also, if you have 3 or more children, it is possible that you should not survive.

In [89]:
print(train_data['SibSp'].describe())

percentages = [0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.93, 0.95, 0.98, 1]

print(train_data['SibSp'].quantile(percentages))

i = 0
while i<9:
    siblings_total = len(train_data.loc[train_data.SibSp==i]["Survived"])
    siblings_survived = sum(train_data.loc[train_data.SibSp==i]["Survived"])
    
    siblings_ratio = 0
    
    if siblings_total != 0:
        siblings_ratio = (siblings_survived/siblings_total)*100

    print(f'From a total of {siblings_total} people with {i} siblings/spouses, {siblings_survived} survived - {siblings_ratio}%')
    i=i+1



**Train/test split**

The goal here is to split our train model in random examples to check if our prediction variables has a high accuracy rate. It include some treatments to transform text data in numeric.

#### Result: 
According to this simulation, the model has an estimated accuracy of 75%.

In [90]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

gender_dict = {'male': 1, 'female': 0}
train_data.Sex = train_data.Sex.map(gender_dict)
train_data.head()

y = train_data['Survived']

X = train_data[["Age", "Pclass", "SibSp", "Sex", "Parch"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)

model = LinearSVC()
model.fit(X_train, y_train)
prediction = model.predict(X_test)

accuracy = accuracy_score(y_test, prediction) * 100
print(f'{accuracy}% accuracy')

gender_dict = {1: 'male', 0: 'female'}
train_data.Sex = train_data.Sex.map(gender_dict)

In [91]:
from sklearn.ensemble import RandomForestClassifier

y = train_data['Survived']

features = ["Pclass", "SibSp", "Age", "Sex","Parch", "Embarked"]

X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

train_data[features].head()
pd.get_dummies(train_data[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)

predictions = model.predict(X_test)
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print('Your submission was successfully saved')